## Plan
1. Obtain data from Kaggle and lyrics from Genius
2. Train Model on genre using audio features and lyrics
3. Load songs and genre into database
4. Create user interactions through discord 
    - Allow songs within database to get genre from there
    - Allow new songs to be run through model
    - Save new songs to databse

In [47]:
import pandas as pd

In [53]:
spotify_2000 = pd.read_csv('data/Spotify-2000.csv', index_col=0)
spotify_tracks = pd.read_csv('data/Spotify-Tracks.csv', index_col=0)
spotify_2000 = spotify_2000[~spotify_2000['Title'].isin(spotify_tracks['Title'])]
spotify_2000 = spotify_2000.drop(columns=['Year','Beats Per Minute (BPM)'])
spotify_tracks = spotify_tracks[spotify_2000.columns]
data = pd.concat([spotify_tracks, spotify_2000], ignore_index=True)
data.to_csv('data/All-Songs.csv')
data

,Title,Artist,Top Genre,Energy,Danceability,Loudness (dB),Liveness,Valence,Length (Duration),Acousticness,Speechiness,Popularity
0,Comedy,Gen Hoshino,acoustic,0.4610,0.676,-6.746,0.3580,0.715,230666,0.0322,0.1430,73
1,Ghost - Acoustic,Ben Woodward,acoustic,0.1660,0.420,-17.235,0.1010,0.267,149610,0.9240,0.0763,55
2,To Begin Again,Ingrid Michaelson;ZAYN,acoustic,0.3590,0.438,-9.734,0.1170,0.120,210826,0.2100,0.0557,57
3,Can't Help Falling In Love,Kina Grannis,acoustic,0.0596,0.266,-18.515,0.1320,0.143,201933,0.9050,0.0363,71
4,Hold On,Chord Overstreet,acoustic,0.4430,0.618,-9.681,0.0829,0.167,198853,0.4690,0.0526,82
...,...,...,...,...,...,...,...,...,...,...,...,...
115011,Midnight Rambler,The Rolling Stones,album rock,66.0000,40.000,-10.000,17.0000,51.000,413,77.0000,12.0000,50
115012,Get Ready,Rare Earth,blues rock,87.0000,41.000,-6.000,83.0000,65.000,"1,292",0.0000,4.0000,45
115013,Black Magic Woman - 2018 Remaster,Fleetwood Mac,album rock,41.0000,46.000,-11.000,28.0000,44.000,173,5.0000,4.0000,48
115014,Suite: Judy Blue Eyes - Remastered,"Crosby, Stills & Nash",album rock,36.0000,45.000,-13.000,20.0000,30.000,444,47.0000,4.0000,67


In [68]:
import sqlite3

con = sqlite3.connect('data/spotify.db')
cursor = con.cursor()
cursor.execute('DROP TABLE IF EXISTS songs')
data.to_sql('songs', con, if_exists='append', index=False)
con.commit()


In [69]:
# Execute the query
rows = cursor.execute("SELECT * FROM songs LIMIT 10")

column_names = [description[0] for description in cursor.description]
print("Column Names:", column_names)

# Print all rows
for row in rows:
    print(row)



Column Names: ['Title', 'Artist', 'Top Genre', 'Energy', 'Danceability', 'Loudness (dB)', 'Liveness', 'Valence', 'Length (Duration)', 'Acousticness', 'Speechiness', 'Popularity']
('Comedy', 'Gen Hoshino', 'acoustic', 0.461, 0.676, -6.746, 0.358, 0.715, '230666', 0.0322, 0.143, 73)
('Ghost - Acoustic', 'Ben Woodward', 'acoustic', 0.166, 0.42, -17.235, 0.101, 0.267, '149610', 0.924, 0.0763, 55)
('To Begin Again', 'Ingrid Michaelson;ZAYN', 'acoustic', 0.359, 0.438, -9.734, 0.117, 0.12, '210826', 0.21, 0.0557, 57)
("Can't Help Falling In Love", 'Kina Grannis', 'acoustic', 0.0596, 0.266, -18.515, 0.132, 0.143, '201933', 0.905, 0.0363, 71)
('Hold On', 'Chord Overstreet', 'acoustic', 0.443, 0.618, -9.681, 0.0829, 0.167, '198853', 0.469, 0.0526, 82)
('Days I Will Remember', 'Tyrone Wells', 'acoustic', 0.481, 0.688, -8.807, 0.189, 0.666, '214240', 0.289, 0.105, 58)
('Say Something', 'A Great Big World;Christina Aguilera', 'acoustic', 0.147, 0.407, -8.822, 0.0913, 0.0765, '229400', 0.857, 0.0355

In [ ]:
# Close the connection
con.close()
